In [1]:
import torch
import torch.nn as nn
from DL_approach.LeNet import LeNet5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Beispiel: GTSRB mit 43 Klassen, aber 32x32 (weil du strict LeNet willst)
model = LeNet5(in_channels=3, num_classes=43, input_size=32, activation="tanh", adapt_to_lenet_geometry=False).to(device)

# Dummy-Batch: batch_size=8, 3 Kanäle, 32x32
x = torch.randn(8, 3, 32, 32, device=device)

# Forward
logits = model(x)
print("logits shape:", logits.shape)   # erwartet: torch.Size([8, 43])

# Loss + Backward (prüft, ob Gradientenfluss ok ist)
y = torch.randint(0, 43, (8,), device=device)
criterion = nn.CrossEntropyLoss()
loss = criterion(logits, y)
loss.backward()

print("loss:", float(loss))
print("backward ok, example grad mean:",
      model.c1.weight.grad.abs().mean().item())

n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("trainable parameters:", n_params)


logits shape: torch.Size([8, 43])
loss: 3.772609233856201
backward ok, example grad mean: 0.0027019288390874863
trainable parameters: 64811


In [2]:
# quick_train_lenet_gtsrb_debug.py
# Minimal debug run: load only a fraction of GTSRB and train LeNet for a few epochs.

import torch
import torch.nn as nn
import torch.optim as optim

from data.gtsrb_loader import get_gtsrb_dataloaders
from DL_approach.train_utils import train_model
from DL_approach.LeNet import LeNet5  # adjust import to your file name


def main():
    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    # Debug fraction: e.g., 5% of train + 5% of test
    debug_fraction = 1 # 0.05

    # Data
    trainloader, testloader = get_gtsrb_dataloaders(
        root="data/GTSRB",
        img_size=(32, 32),          # strict classic LeNet geometry
        batch_size=128,
        num_workers=0,
        normalize=True,
        debug_fraction=debug_fraction,
        seed=42,
    )

    print("Train samples:", len(trainloader.dataset))
    print("Test  samples:", len(testloader.dataset))

    # Model
    model = LeNet5(
        in_channels=3,
        num_classes=43,
        input_size=32,
        activation="tanh",
        adapt_to_lenet_geometry=False,
        dropout_p=0.2,              # set 0.0 to disable
    ).to(device)

    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Train (uses testloader as eval loader here, matching your preferred workflow)
    run_tag = f"GTSRB32_dbg{debug_fraction}_LeNet5_tanh_adam1e-3_do0.2"
    model, history = train_model(
        model=model,
        trainloader=trainloader,
        evalloader=testloader,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        epochs=3,
        save_curves=True,
        curves_dir="training_curves",
        run_tag=run_tag,
    )

    # Final eval (optional explicit call)
    model.eval()
    print("\nDone.")


if __name__ == "__main__":
    main()


Device: cuda
Train samples: 39209
Test  samples: 12630

Epoch 1/3


KeyboardInterrupt: 

In [4]:
# quick_train_lenet_gtsrb_debug.py
# Minimal debug run: load only a fraction of GTSRB and train LeNet for a few epochs.

import torch
import torch.nn as nn
import torch.optim as optim

from data.cifar_loader import CIFAR10
from DL_approach.train_utils import train_model
from DL_approach.LeNet import LeNet5  # adjust import to your file name


def main():
    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    # Debug fraction: e.g., 5% of train + 5% of test
    debug_fraction = 1 # 0.05

    # Data
    trainloader, testloader = get_gtsrb_dataloaders(
        root="data/GTSRB",
        img_size=(32, 32),          # strict classic LeNet geometry
        batch_size=128,
        num_workers=0,
        normalize=True,
        debug_fraction=debug_fraction,
        seed=42,
    )

    print("Train samples:", len(trainloader.dataset))
    print("Test  samples:", len(testloader.dataset))

    # Model
    model = LeNet5(
        in_channels=3,
        num_classes=43,
        input_size=32,
        activation="tanh",
        adapt_to_lenet_geometry=False,
        dropout_p=0.2,              # set 0.0 to disable
    ).to(device)

    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Train (uses testloader as eval loader here, matching your preferred workflow)
    run_tag = f"GTSRB32_dbg{debug_fraction}_LeNet5_tanh_adam1e-3_do0.2"
    model, history = train_model(
        model=model,
        trainloader=trainloader,
        evalloader=testloader,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        epochs=25,
        save_curves=True,
        curves_dir="training_curves",
        run_tag=run_tag,
    )

    # Final eval (optional explicit call)
    model.eval()
    print("\nDone.")


if __name__ == "__main__":
    main()


Device: cuda
Train samples: 39209
Test  samples: 12630

Epoch 1/25


Train Loss: 1.6276 | Train Acc: 0.5649 | Time: 21.09s
Eval  Loss: 0.7348 | Eval  Acc: 0.8055 | Time: 7.50s

Epoch 2/25


Train Loss: 0.3810 | Train Acc: 0.9091 | Time: 27.51s
Eval  Loss: 0.4342 | Eval  Acc: 0.8841 | Time: 5.94s

Epoch 3/25


Train Loss: 0.1943 | Train Acc: 0.9557 | Time: 30.91s
Eval  Loss: 0.3788 | Eval  Acc: 0.9000 | Time: 6.81s

Epoch 4/25


Train Loss: 0.1251 | Train Acc: 0.9723 | Time: 23.40s
Eval  Loss: 0.3417 | Eval  Acc: 0.9116 | Time: 4.61s

Epoch 5/25


...

Train Loss: 0.0092 | Train Acc: 0.9977 | Time: 20.18s
Eval  Loss: 0.3973 | Eval  Acc: 0.9211 | Time: 6.84s

Epoch 22/25


Train Loss: 0.0103 | Train Acc: 0.9973 | Time: 19.05s
Eval  Loss: 0.3828 | Eval  Acc: 0.9243 | Time: 5.72s

Epoch 23/25


Train Loss: 0.0094 | Train Acc: 0.9975 | Time: 26.34s
Eval  Loss: 0.3658 | Eval  Acc: 0.9250 | Time: 6.40s

Epoch 24/25


Train Loss: 0.0088 | Train Acc: 0.9979 | Time: 23.42s
Eval  Loss: 0.3884 | Eval  Acc: 0.9252 | Time: 7.03s

Epoch 25/25


Train Loss: 0.0032 | Train Acc: 0.9995 | Time: 22.93s
Eval  Loss: 0.3696 | Eval  Acc: 0.9272 | Time: 5.03s

Runtime summary:
  total train time: 581.95s
  total eval  time: 154.50s
Saved training curves to: training_curves\LeNet5_GTSRB32_dbg1_LeNet5_tanh_adam1e-3_do0_2_20260125-070340_curves.png

Done.


In [ ]:
# quick_train_lenet_cifar_debug.py
# Minimal debug run: use YOUR existing CIFAR10 implementation + dataloader helper
# and train LeNet for a few epochs.

import torch
import torch.nn as nn
import torch.optim as optim

# adjust these imports to your project structure
from data.cifar_loader import get_cifar10_dataloaders   # <- this is the function you posted
from DL_approach.train_utils import train_model
from DL_approach.LeNet import LeNet5


def main():
    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    # Data
    # - root must be the folder that contains "cifar-10-batches-py/"
    #   e.g. root="data" if you have data/cifar-10-batches-py/...
    trainloader, testloader = get_cifar10_dataloaders(
        root="data",
        batch_size=128,
        num_workers=0, 
        normalize=True,
        img_size=(32, 32),
        train_transform=None,
        test_transform=None,
    )

    print("Train samples:", len(trainloader.dataset))
    print("Test  samples:", len(testloader.dataset))

    # Model (CIFAR-10 -> 10 classes)
    model = LeNet5(
        in_channels=3,
        num_classes=10,
        input_size=32,
        activation="tanh",
        adapt_to_lenet_geometry=False,
        dropout_p=0.2,
    ).to(device)

    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Train (using testloader as eval loader, same as your GTSRB debug script)
    run_tag = "CIFAR10_32_dbg_LeNet5_tanh_adam1e-3_do0.2"
    model, history = train_model(
        model=model,
        trainloader=trainloader,
        evalloader=testloader,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        epochs=30,
        save_curves=True,
        curves_dir="training_curves",
        run_tag=run_tag,
    )

    # Quick sanity eval
    model.eval()
    with torch.no_grad():
        xb, yb = next(iter(testloader))
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        preds = logits.argmax(dim=1)
        acc = (preds == yb).float().mean().item()
        print(f"One-batch test accuracy: {acc:.3f}")

    print("\nDone.")


if __name__ == "__main__":
    main()


Device: cuda
Train samples: 50000
Test  samples: 10000

Epoch 1/30


Train Loss: 1.7972 | Train Acc: 0.3587 | Time: 4.57s
Eval  Loss: 1.6284 | Eval  Acc: 0.4244 | Time: 0.44s

Epoch 2/30


Train Loss: 1.5444 | Train Acc: 0.4488 | Time: 3.95s
Eval  Loss: 1.4457 | Eval  Acc: 0.4855 | Time: 0.50s

Epoch 3/30


Train Loss: 1.3990 | Train Acc: 0.5003 | Time: 3.62s
Eval  Loss: 1.3606 | Eval  Acc: 0.5159 | Time: 0.56s

Epoch 4/30


Train Loss: 1.3165 | Train Acc: 0.5295 | Time: 7.13s
Eval  Loss: 1.2969 | Eval  Acc: 0.5346 | Time: 0.99s

Epoch 5/30


...

Train Loss: 0.8151 | Train Acc: 0.7132 | Time: 5.05s
Eval  Loss: 1.3355 | Eval  Acc: 0.5614 | Time: 1.06s

Epoch 27/30


Train Loss: 0.7991 | Train Acc: 0.7180 | Time: 5.19s
Eval  Loss: 1.3416 | Eval  Acc: 0.5631 | Time: 0.53s

Epoch 28/30


Train Loss: 0.7912 | Train Acc: 0.7218 | Time: 4.57s
Eval  Loss: 1.3458 | Eval  Acc: 0.5626 | Time: 0.51s

Epoch 29/30


Train Loss: 0.7771 | Train Acc: 0.7256 | Time: 4.40s
Eval  Loss: 1.3920 | Eval  Acc: 0.5561 | Time: 0.54s

Epoch 30/30


Train Loss: 0.7709 | Train Acc: 0.7285 | Time: 5.69s
Eval  Loss: 1.3764 | Eval  Acc: 0.5606 | Time: 0.53s

Runtime summary:
  total train time: 148.02s
  total eval  time: 18.84s
Saved training curves to: training_curves\LeNet5_CIFAR10_32_dbg_LeNet5_tanh_adam1e-3_do0_2_20260125-061400_curves.png
One-batch test accuracy: 0.566

Done.
